In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow import HornSchunck, getimgfiles
from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
def horn_schunck(tensor, frames=None):
    if not frames:
        frames = len(tensor)-1
    x_comp = []
    y_comp = []
    for x in range(frames):
        U, V = HornSchunck(tensor[x,:,:], tensor[x+1,:,:], alpha=1.0, Niter=100)
        x_comp.append(U)
        y_comp.append(V)
        print(".",end="")
    return np.array(x_comp), np.array(y_comp)

def display_combined(u, v, Inew, scale = 100, quivstep = 3, fig=None, ax=None, figsize=(10,10)):
    if not fig:
        fig, ax = plt.subplots(1,figsize=figsize)
    ax.cla()
    im = ax.imshow(Inew, vmin=.1,vmax=.2)

    for i in range(0, u.shape[0], quivstep):
        for j in range(0, v.shape[1], quivstep):
            ax.arrow(j, i, v[i, j] * scale, u[i, j] * scale, color='red', head_width=0.5, head_length=1,)
            
    return fig, ax

arrows = pickle.load(open("arrows.pkl","rb"))
def quiver_quick(background_raw, x_comp, y_comp, block_size=16):
    global arrows
    background = np.ndarray([background_raw.shape[0],background_raw.shape[1],3],dtype=np.uint8)
    background[:,:,0] = background_raw
    background[:,:,1] = background_raw
    background[:,:,2] = background_raw


    
    horizontal_blocks = y_comp.shape[1]//block_size
    horizontal_indent = (y_comp.shape[1]%block_size)//2
    x_mins = range(horizontal_indent,horizontal_indent+horizontal_blocks*block_size,block_size)
    
    vertical_blocks = y_comp.shape[0]//block_size
    vertical_indent = (y_comp.shape[0]%block_size)//2
    y_mins = range(vertical_indent,vertical_indent+vertical_blocks*block_size,block_size)
    
    
    block = np.ones((block_size,block_size))
    for y in y_mins:
        for x in x_mins:
            angle = (np.round(np.rad2deg(np.arctan2(x_comp[y,x],y_comp[y,x])/10))*10)

            if angle >= 360:
                angle -=360
            elif angle <0:
                angle += 360
            
            bg = background[y:y+block_size,x:x+block_size]
            arrow = arrows[block_size][angle]
            nonzero = arrow>0
            bg[:,:,0][nonzero] = arrow[arrow>0]
            bg[:,:,1][nonzero] -= arrow[arrow>0]
            bg[:,:,2][nonzero] -= arrow[arrow>0]

            #= arrows[block_size][angle]


    return background  

def normalize(frames):
    frames -= np.min(frames)
    frames /= np.max(frames)
    return frames

def normal_difference(frames, mean):
    frames = frames - mean
    return normalize(frames)

def apply_mask(frames, mask):
    for i, f in enumerate(frames):
        f[mask] = 0
        frames[i] = f
    return frames

def substract_pixel_min(tensor):
    for y in range(tensor.shape[1]):
        for x in range(tensor.shape[2]):
            tensor[:,y,x] -= np.min(tensor[:,y,x])
    return tensor

def maxima(vector, pre_smoothing=100, minval=0):
    """ Returns indices of local maxima sorted by value at each indice starting with the highest value
    args:
        vector: Vector of data
        pre_smoothing: Set high values to detect substantial peaks only.
    returns: List of local maxima
    """
    extrema = None
    if pre_smoothing > 0:
        extrema = argrelextrema(gaussian_filter(vector,pre_smoothing),np.greater)[0]
    else:
        extrema = argrelextrema(vector,np.greater)[0]
    vals = vector[extrema].flatten()
    return extrema[vals>minval]

# Similation of dense optical flow for growing focus of activation

In [ ]:
tensor = np.zeros((50,100,100))
tensor[20,20:60,20:60] = 1
tensor = normalize(gaussian_filter(tensor,10))
x_comp_sim, y_comp_sim = horn_schunck(tensor)

In [ ]:
tensor.shape

In [ ]:
x_comp_sim.shape

In [ ]:
%%capture
fig_sim, ax_sim = display_combined(x_comp_sim[0],y_comp_sim[0], tensor[0])
start = 0
frames = 40

def animate(i):
    global start, x_comp_sim, y_comp_sim, tensor
    i += start
    print(".", end ="")    
    display_combined(x_comp_sim[i], y_comp_sim[i], tensor[i+1], fig=fig_sim, ax=ax_sim)

ani_sim = matplotlib.animation.FuncAnimation(fig_sim, animate, frames=frames)

In [ ]:
plt.imshow(x_comp_sim[28]+y_comp_sim[28])
plt.colorbar()

In [ ]:
from IPython.display import HTML
HTML(ani_sim.to_jshtml())

Motion vectors indicate that there is movement in the opposite direction of the gradient.

# Test quick vector plot

In [ ]:
mat = np.ones((100,100))
x_comp_test = np.ones((100,100))
y_comp_test = np.ones((100,100))
y_comp_test *= .0001
x_comp_test *= .0001
y_comp_test[0:50,0:50] = 1*.0001

In [ ]:
mat = np.ones((100,100))
x_comp_test = -np.ones((100,100))
y_comp_test = -np.ones((100,100))
y_comp_test *= .0001
x_comp_test *= .0001
y_comp_test[0:50,0:50] = 1*.0001

In [ ]:
frame = 40
plt.imshow(quiver_quick(mat.copy()*255,x_comp_test, y_comp_test,20))

In [ ]:
f, a = plt.subplots(1, figsize=(5,5))
display_combined(x_comp_test, y_comp_test, mat, fig=f, ax=a)

In [ ]:
frame = 20
plt.imshow(quiver_quick(tensor[frame,:,:].copy()*128,x_comp_sim[frame,:,:], y_comp_sim[frame,:,:],10))

In [ ]:
f, a = plt.subplots(1, figsize=(6,6))
display_combined(x_comp_sim[frame,:,:], y_comp_sim[frame,:,:], tensor[frame,:,:].copy()*255, fig=f, ax=a)

# Inspect a frame of the raw data

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

In [ ]:
frames = skimage.io.imread(os.path.join(source_folder,"runstart16_X1.tif"))

In [ ]:
plt.imshow(frames[0,:,:])

In [ ]:
frames.shape

In [ ]:
frames = frames[:2000,:,:]

# Preprocessing

Here I calculate the difference from pixelwise mean as well as a smoothed version that promised to increase the signal to noise ratio.

In [ ]:
mean = np.mean(frames,axis=0)#pixelwise mean

In [ ]:
difference = normal_difference(frames, mean)

In [ ]:
smooth = normalize(gaussian_filter(substract_pixel_min(difference), 1))

In [ ]:
smoother = normalize(gaussian_filter(substract_pixel_min(difference), 5))

In [ ]:
details = smooth-smoother

# Inspect the mean of frames in time

- Is there a systematic error/trend in the timeseries?

- Could the short wave events be identified?

In [ ]:
frame_means = [np.mean(f) for f in smooth]
frame_max = [np.max(f) for f in smooth]
upper_decentile = [np.quantile(f,0.9) for f in smooth]
smooth_max = gaussian_filter(frame_max,10)
smoother_max = gaussian_filter(frame_max,20)

In [ ]:
maxs = maxima(np.array(smoother_max),0,.1)

In [ ]:
np.max(maxs)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,figsize=(10,10))
ax.plot(frame_means, label="mean")
ax.plot(frame_max, label="max")
ax.plot(upper_decentile, label = "upper decentile")
ax.plot(smoother_max, label="smooth max")
ax.legend(loc="upper left")
ax.set_xlabel("frame")
ax.set_ylabel("value")
for x in maxs:
    ax.axvline(x, c="lightgray")

In [ ]:
def count_peaks(vector, threshold):
    vector = np.array(vector) > threshold
    prev_val = vector[0]
    count = 0
    for x in vector[1:]:
        if prev_val != x:#If previous value was different (e.g. False and now we have True)
            count += 1
            prev_val = x
    
    return (count)//2

In [ ]:
peaks_for_thresholds = [count_peaks(smooth_max,x) for x in np.arange(100)/100]
plt.plot(np.linspace(0,1,100),peaks_for_thresholds)
print(np.where(peaks_for_thresholds==np.max(peaks_for_thresholds)))


In [ ]:
plt.scatter(upper_decentile, frame_max)

There is no systematic error (trend) for the timeseries of activation (luminance difference from mean).

The upper decentile and the maximal value correlate strongly. The images are not affected by single pixel outliers (non-gaussian noise). Hence, a strategy of detecting slow-wave events based on (local maxima of) maximal values appears feasable.

# Inspect smooth version and determine feasable thresholds

For visualization of the slow waves total activation

In [ ]:
%%capture
fig, ax = plt.subplots(1, figsize=(10,10))

im = ax.imshow(smooth[0,:,:], vmin =.1, vmax=.2)#vmin=.25,vmax=.3)
startframe = 50
ani = matplotlib.animation.FuncAnimation(fig, lambda i: im.set_array(smooth[startframe+i]), frames=150).to_jshtml()

For visualization of nuances of small scale travelling peaks in the activation by linear scaling mapping the lowest value to 0 and the highest to 1. 

In [ ]:
%%capture
import matplotlib.animation
from IPython.display import HTML
fig, ax = plt.subplots(1, figsize=(10,10))
def display(frame):
    global fig, ax
    ax.cla()
    im = ax.imshow(normalize(frame),vmin=0,vmax=1)
    return fig, ax
startframe = 50
ani = matplotlib.animation.FuncAnimation(fig, lambda i: display(smooth[startframe+i]), frames=150).to_jshtml()

In [ ]:
%%capture
import matplotlib.animation
from IPython.display import HTML
fig, ax = plt.subplots(1, figsize=(10,10))
def display(frame):
    global fig, ax
    ax.cla()
    im = ax.imshow(frame,vmin=.0,vmax=1)
    return fig, ax
startframe = 50
ani = matplotlib.animation.FuncAnimation(fig, lambda i: display(details[startframe+i]), frames=150).to_jshtml()

In [ ]:
HTML(ani)

# Horn and Schunck dense optical flow

In [ ]:
x_comp, y_comp = horn_schunck(details, 200)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 50
frames = 100

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(x_comp[i],y_comp[i], details[i+1], fig=fig, ax=ax)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

# Check for maximal values

In [ ]:
idxs = maxs#maxima(np.mean(np.mean(smooth,axis=1),axis=1),2,.15)

In [ ]:
frame = smooth[idxs[3]]
plt.imshow(frame, vmin=.1, vmax=.5)
max_loc = np.where(frame == np.max(frame))
plt.scatter(max_loc[1],max_loc[0], marker="x", c="red")

In [ ]:
xs = []
ys = []
for idx in idxs:
    frame = smooth[idx]
    max_loc = np.where(frame == np.max(frame))
    xs.extend(max_loc[1])
    ys.extend(max_loc[0])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,20))
ax[0].axis('off')
ax[0].imshow(mean)
ax[0].scatter(xs, ys, marker="x", c="red")
ax[1].axis('off')

Looks like secondary visial cortices and motor cortices.

# Approach 2

Assumption: The earliest detectable region of a beginning peak in activation relates to it's origin 

Additional assumption: If there is more then one region even at the beginning of neural activity one could additionally assume that the area with the strongest activation is the source

In [ ]:
import numpy as np
import scipy.stats as st

class FastDensityClustering():
    @staticmethod
    def gaussian_kernel(size=21, nsig=3):
        """Returns a 2D Gaussian kernel.
        Args:
            size: The size of the kernel (size x size)
            nsig: Sigma of the gaussian
        """
        x = np.linspace(-nsig, nsig, size+1)
        kern1d = np.diff(st.norm.cdf(x))
        kern2d = np.outer(kern1d, kern1d)
        return kern2d/kern2d.sum()

    @staticmethod
    def kernel(size, ktype):
        """ Returns a kernel of specified size and type
        Args:
            size: Kernel size
            ktype: Type of kernel. Either uniform gaussian or disk are provided.
        """
        if ktype == "uniform":
            return np.ones((size,size))
        elif ktype == "gaussian":
            k = FastDensityClustering.gaussian_kernel(size=size)
            k /= np.max(k)
            return k
        elif ktype == "disk":
            k = FastDensityClustering.gaussian_kernel(size=size)
            k /= np.max(k)
            return k > 0.03

    @staticmethod
    def collapse_iteration(arr,kernel):
        """ Determins center of gravity for each non-zero (forground pixel) and it's surround weighted by the kernel
            and increases mass at named target position/pixel by the mass of the source pixel.
        Args:
            arr: Grayscale array of positive values where value zero stands for the background and positive values denote the mass for a given foreground pixel.
            kernel: Kernel used to weight the influance of nearby pixels in computing the center of mass
        """
        kernel_width = kernel.shape[0]
        kernel_height = kernel.shape[1]
        ys, xs = np.where(arr>0)
        new = np.zeros(arr.shape)
        abs_shift = 0

        mapping = {}
        for y, x in zip(ys,xs):
            snippet = arr[y-kernel_width//2:(y+kernel_width//2)+1, x-kernel_width//2:(x+kernel_width//2)+1]

            snippet = kernel * snippet
            weights_x = np.mean(snippet,axis=0)
            weights_y = np.mean(snippet,axis=1)

            shift_x = np.average(np.arange(kernel_width),weights=weights_x)#The inner mean returns x values, the outer is their mean -> shift x
            shift_y = np.average(np.arange(kernel_height),weights=weights_y)#The inner mean returns y values, the outer is their mean -> shift y
            shift_x -= (kernel_width-1)/2
            shift_y -= (kernel_height-1)/2


            y1 = int(y+shift_y)
            x1 = int(x+shift_x)

            #Remember where the contribution of the mass of the tatget comes from
            if y1 != y or x1 != x:
                if not str([y1,x1]) in mapping:
                    mapping[str([y1,x1])] = []
                mapping[str([y1,x1])] = [y,x]

            abs_shift += np.abs(shift_x) + np.abs(shift_y)
            new[y1,x1] += arr[y,x]
        if len(xs) > 0:
            shift = abs_shift/len(xs)
        else:
            shift = 0
        return new, shift, mapping

    @staticmethod
    def collapse(arr, iterations = None,gravity_type="uniform", gravity_size=5):
        """ Performs clustering by iteratively moving all mass densities (non-zero/foreground pixels) to their center of mass.
        If no value for iterations is specified the algorithm runs until convergence is achieved and the movement is marginally.
        Args:
            arr: Array of positive gray values
            iterations: Number of iterations. If no value for iterations is specified the algorithm runs until convergence is achieved.
            gravity_type: Either "uniform", "gaussian" or "disk". The contributions to the center of mass for one pixels by its surround are weighted accordingly.
            gravity_size: The size of the gravity kernel.
        Returns:
            Array representation of cluster centers. Each cluster center is represented by a non-zero pixel.
        """
        epsilon = None
        if not iterations:
            iterations = 1000
            epsilon = 1.0e-16

        if gravity_size % 2 == 0:
            gravity_size += 1
        k = FastDensityClustering.kernel(gravity_size,gravity_type)
        arr = np.pad(arr,gravity_size, "constant")
        mappings = []
        for x in range(iterations):
            arr, shift, mapping = FastDensityClustering.collapse_iteration(arr,k)
            mappings.append(mapping)
            if epsilon:
                if epsilon > shift:
                    break


        return arr[gravity_size:-gravity_size,gravity_size:-gravity_size], mappings

    @staticmethod
    def density_clustering(arr, iterations = None, gravity_type="uniform", gravity_size=5):
        """ Performs clustering by iteratively moving all mass densities (non-zero/foreground pixels) to their center of mass.
        If no value for iterations is specified the algorithm runs until convergence is achieved and the movement is marginally.
        Args:
            arr: Array of positive gray values
            iterations: Number of iterations. If no value for iterations is specified the algorithm runs until convergence is achieved.
            gravity_type: Either "uniform", "gaussian" or "disk". The contributions to the center of mass for one pixels by its surround are weighted accordingly.
            gravity_size: The size of the gravity kernel.
        Returns:
            Y and x positions of all detected cluster centers
        """
        cluster_array, mappings = FastDensityClustering.collapse(arr,iterations,gravity_type=gravity_type, gravity_size=gravity_size)
        center_y,center_x = np.where(cluster_array>0)
        print(".",end="")
        return center_y, center_x, cluster_array, mappings


## Mask out values based on a threshold

## Find frames that correspond to early stages of spreading neural activation

In [ ]:
thresholded = smooth.copy()
thresholded[thresholded <.15] = 0
thresholded = normalize(thresholded)

In [ ]:
mean_thresholded = np.mean(np.mean(thresholded, axis=1),axis=1)
maxs_thresholded = maxima(np.mean(np.mean(smooth,axis=1),axis=1),2,.15)

event_start_idxs = []
event_active = False
for i, v in enumerate(mean_thresholded>.001):
    if v and not event_active:
        event_active = True
        event_start_idxs.append(i)
    if not v:
        event_active = False

In [ ]:
idx_max = 600

fig, ax = plt.subplots(1)
plt.plot(mean_thresholded[:idx_max])
plt.plot((mean_thresholded[:idx_max]>.001)/10)
for x in event_start_idxs:
    if x > idx_max:
        continue
    ax.axvline(x, c="lightgray")

## Use fast density clustering to find clustercenters of strongest activation

In [ ]:
idx = 13

fig, ax = plt.subplots(1)
frame = normalize(thresholded[event_start_idxs[idx]])>.95
res = FastDensityClustering.density_clustering(frame,gravity_size=40)
ax.imshow(smooth[event_start_idxs[idx]], vmin=.0, vmax=.5)
ax.scatter(np.where(frame)[1],np.where(frame)[0])
ax.scatter(res[1],res[0], marker="x", color="red")


### With additional assumption (strongest activation cluster is origin)

In [ ]:
ys = []
xs = []
for event_start_idx in event_start_idxs:
    frame = normalize(thresholded[event_start_idx])>.95
    res = FastDensityClustering.density_clustering(frame,gravity_size=10)
    center_density = res[2][res[:2]]#The density of the points that collapsed to named cluster center
    heviest_center = np.where(center_density == np.max(center_density))      
    ys.extend(res[0][heviest_center])
    xs.extend(res[1][heviest_center])
    #ys.extend(res[0])
    #xs.extend(res[1])

In [ ]:
plt.imshow(mean)
plt.scatter(xs, ys, marker="x", c="red")

### Without additional assumption

In [ ]:
ys = []
xs = []
for event_start_idx in event_start_idxs:
    frame = normalize(thresholded[event_start_idx])>.95
    res = FastDensityClustering.density_clustering(frame,gravity_size=10)
    ys.extend(res[0])
    xs.extend(res[1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,20))
ax[0].axis('off')
ax[0].imshow(mean)
ax[0].scatter(xs, ys, marker="x", c="red")
ax[1].axis('off')
ax[1].imshow(Image.open("cortex.bmp"))

Looks again like the source of activation stems from secondary visial cortices and motor cortices. 

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,15))
ax.axis('off')
ax.imshow(Image.open("with_boundaries.png"))

Possible analysis with ANNs
- Predict location based on history of locations for previous events using regression MLPs. Are there regulrities?
- Predict next frame with previous frame using autoencoder?
- Predict experimental condition using some characteristic of the signal slope (e.g. mean per frame)?
- Go into direction of spiking neural networks/ biological models for neural networks?
- Use information about anatomical connectivity & initial activation to predict activation of later frames. Compare to prediction using initial activation only. Could the prediction be improved when considering anatomical conncectivity?

# Sources

- Kirkcaldie, M. T. K., Watson, C., Paxinos, G., & Franklin, K. (2012). Straightening out the mouse neocortex. In Australian Neuroscience Society Annual Conference. Available online via https://www.researchgate.net/profile/Matthew_Kirkcaldie/publication/234062488_Straightening_out_the_mouse_neocortex/links/09e4150ef5c5a1214d000000/Straightening-out-the-mouse-neocortex.pdf.
